In [ ]:
# to reload modules when cell is run
%load_ext autoreload
%autoreload 2

import sys
# add to path
sys.path.append(r"C:\Users\oliver.koehn\Documents\projects\lodsApp")
sys.path.append(r"C:\Users\oliver.koehn\Documents\projects\lodsApp\lib")

from lib.utils import *
from lib.nn import *

import json
from sklearn.model_selection import GridSearchCV

# regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

# classification
from sklearn.ensemble import RandomForestClassifier

testcase = "berka"
#testcase = "income"
#testcase = "stars"
#testcase = "realEstate"
#testcase = "medicalCost"
testcase = "autoMpg"

with open("config_tests.json", "r") as f:
    j = json.load(f)[testcase]

session = j["session"]

In [ ]:
## TRAIN NETWORK

################################################################################
# we need the df to get default session parameters
j__df = get_j__df(session['fp_tmp_user_num'], session['delimiter'], n__sample = 3)
textfields_frontend = get_frontend_default_columnNames_columnTypes_joins(j__df)
session.update(textfields_frontend)
################################################################################

#################################################################################################################################################
########################## GET DF ########################################################################################################
#################################################################################################################################################

df = get_df(session)
#################################################################################################################################################
########################## SET TRAIN INPUTS ########################################################################################################
#################################################################################################################################################

# get input cols
df__num, df__cat = get_input_cols(df.copy(), session["tar_col"], th_num = 0, th_cat = 0)
session["num_cols"] = [e for e in df__num["Column Name (Numerical)"].to_list() if e not in [session["tar_col"]]]
session["cat_cols"] = [e for e in df__cat["Column Name (Categorical)"].to_list() if e not in [session["tar_col"]]]

#################################################################################################################################################
########################## TRAIN NETWORK ########################################################################################################
#################################################################################################################################################

session["predict_type"] = set_classification_type(df, session)

if session["predict_type"].lower() == "classification":
    list__methods=[RandomForestClassifier]
    session["main_metric"] = "f1_score"
    session["best_type"] = "MAX"
else:
    list__methods=[LinearRegression, RandomForestRegressor, MLPRegressor]
    session["main_metric"] = "nmse"
    session["best_type"] = "MIN"

pipeline, metrics, result_visualization, res, res_with_best_metric = train_network(df, session, "dummy", list__methods=list__methods, param_grid=None)
    


In [ ]:
result_visualization

In [ ]:
res_with_best_metric[1]["metric"]["test"]

In [ ]:
res

In [ ]:
# USE TRAINED NETWORK

# get pipeline
session = {}
fp_network = r"data\num_networks\dummy.pickle"
with open(fp_network, 'rb') as f:
        j = pickle.load(f)
pipeline = j["pipeline"]
session.update(j["session"])

# read data
df = get_df(session)

# use pipeline on data
p = pipeline.predict(df)
df["predicted"] = p

df

In [ ]:
j